In [ ]:
import pandas as pd
import os
import numpy as np
import shutil
from distutils.dir_util import copy_tree
from transformers import RobertaTokenizer


In [ ]:
# Merge folders into one: Training set
nb_folders = 9

all_path = "../data_handwritten"
if os.path.exists(all_path):
    shutil.rmtree(all_path)
os.mkdir(all_path)
df = pd.DataFrame(columns=["path", "text"])

for folder_id in range(1,nb_folders+1):
    print(folder_id)
    path_original_folder = "data_handwritten_" + str(folder_id)
    os.mkdir(os.path.join(all_path, path_original_folder))
    df_original = pd.read_csv(os.path.join("..", path_original_folder, "train.csv"))
    df_original["path"] = path_original_folder + "/" + df_original["path"]
    df = pd.concat([df, df_original], ignore_index=True)
    copy_tree(os.path.join("..", path_original_folder, "train"), os.path.join(all_path, path_original_folder))
df.to_csv(os.path.join(all_path, "train.csv"), index=False)

In [ ]:
# Small modification to the test set to fix the paths
# Caution, only run once!
df_original = pd.read_csv(os.path.join("..", "data_handwritten_test", "test.csv"))
df_original["path"] = "test/" + df_original["path"]
df_original.to_csv(os.path.join("..", "data_handwritten_test", "test.csv"), index=False)

In [ ]:
# Preprocess the dataframe to save time afterwards
from transformers import RobertaTokenizer

set_ = "test" # train or test

if set_ == "test":
    path = "data_handwritten_test"
elif set_ == "train":
    path = "data_handwritten"
tokenizer = RobertaTokenizer.from_pretrained('PlanTL-GOB-ES/roberta-base-bne')
df = pd.read_csv(os.path.join("..", path, set_+".csv"))
df["tokenizer_out"] = df["text"].apply(lambda x: tokenizer(x))
df["tokens"] = df["tokenizer_out"].apply(lambda x: x["input_ids"])
df["attention_mask"] = df["tokenizer_out"].apply(lambda x: x["attention_mask"])
df["token_length"] = df["tokens"].apply(lambda x: len(x))
df.drop(columns=["tokenizer_out"], inplace=True)
df = df.loc[df["token_length"] <= 512] # This should be fixed in the data_generator: when there are too many tokens, not save.
# df.drop_duplicates(subset=["path"], inplace=True) # This should be fixed in data_generator: for some reason we get many duplicates: UPDATE: Solved
df.to_csv(os.path.join("..", path, set_+"_tokens.csv"), index=False)